In [1]:
import spotipy
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from spotipy.oauth2 import SpotifyClientCredentials
from sklearn.metrics import pairwise_distances_argmin_min

In [2]:
# When you have access to spotify you can get your own songs:

#Initialize SpotiPy with user credentias
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id="af3a4e21d9974f798b0ddef081728f2b",
                                                           client_secret="99a65d20eff04d64bcf24b11824dffc4"))

results = sp.search(q='artist:Justin Bieber', limit=50)


In [3]:
# extract the track ids
track_ids = [track['id'] for track in results['tracks']['items']]

# extract the audio features
audio_features = sp.audio_features(track_ids)

# store audio features in a dataframe
df = pd.DataFrame(audio_features)

In [4]:
# create a function to automate work
def get_audio_features(artist):
    # get tracks from artist
    results = sp.search(q=f'artist:{artist}', limit=50)
    # extract the track ids
    track_ids = [track['id'] for track in results['tracks']['items']]
    song_names = [track['name'] for track in results['tracks']['items']]
    # extract the audio features
    audio_features = sp.audio_features(track_ids)
    # store audio features in a dataframe
    df = pd.DataFrame(audio_features)
    df['artist'] = artist
    df['song name'] = song_names
    return df

In [5]:
# create loop and select artists you want to get songs from

artists = ['Justin Bieber', 'Nirvana', 'pink floyd']

df = pd.DataFrame()

for artist in artists:
    df_artist = get_audio_features(artist)
    df = pd.concat([df, df_artist])
    
df = df.reset_index(drop=True)

In [6]:
x = df[['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo']]
# y = df['artist']

scaler = StandardScaler()

x_prep = scaler.fit_transform(x)

kmeans = KMeans(n_clusters=3, random_state=42)
kmeans.fit(x_prep)

clusters = kmeans.predict(x_prep)

scaled_df = pd.DataFrame(x_prep, columns=x.columns)
scaled_df['song name'] = df['song name']
scaled_df['artist'] = df['artist']
scaled_df['cluster'] = clusters
scaled_df

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,song name,artist,cluster
0,0.531325,0.727921,-0.743235,0.895524,0.816497,-0.293917,-0.295815,-0.539941,0.919154,0.215670,1.089667,Ghost,Justin Bieber,2
1,1.003688,0.543209,-1.264194,0.782752,0.816497,0.927828,0.178234,-0.540051,0.943981,0.312954,-1.149478,Peaches (feat. Daniel Caesar & Giveon),Justin Bieber,0
2,0.549971,0.727921,-0.743235,0.895155,0.816497,-0.299064,-0.320214,-0.539919,0.889363,0.351022,1.089212,Ghost,Justin Bieber,2
3,0.885598,0.288717,-1.003714,0.853511,-1.224745,-0.405452,-0.903712,-0.540051,-0.486022,-0.423024,0.600018,Hold On,Justin Bieber,2
4,0.885598,0.288717,-1.003714,0.853511,-1.224745,-0.405452,-0.903712,-0.540051,-0.486022,-0.423024,0.600018,Hold On,Justin Bieber,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,-0.742811,-0.913963,0.559163,-0.812090,0.816497,-0.594205,0.397829,1.610103,-0.545606,-1.023650,-1.703818,Marooned,pink floyd,1
146,0.344866,-1.500936,0.559163,-1.195920,0.816497,-0.609648,0.575598,-0.066866,0.159465,-0.270753,0.216285,The Show Must Go On,pink floyd,1
147,-1.389203,-0.450131,1.080122,-0.822040,0.816497,-0.571898,0.352516,1.386760,2.428602,-1.205530,0.983051,In the Flesh,pink floyd,1
148,-1.190313,-1.862150,1.080122,-1.723112,-1.224745,-0.511840,-0.645425,2.401269,1.942004,-1.483425,0.023998,Don't Leave Me Now,pink floyd,1


In [7]:
# analyzing our results, we can start to see some winners
scaled_df.groupby(['cluster', 'artist'], as_index=False).count().sort_values(['cluster', 'key'], ascending=[True, False])[['artist', 'cluster', 'key']].reset_index(drop=True)

,artist,cluster,key
0,Justin Bieber,0,25
1,Nirvana,0,8
2,pink floyd,0,6
3,pink floyd,1,39
4,Justin Bieber,1,7
5,Nirvana,1,5
6,Nirvana,2,37
7,Justin Bieber,2,18
8,pink floyd,2,5


Check out how to visualize your clusters [here](https://towardsdatascience.com/how-to-build-an-amazing-music-recommendation-system-4cce2719a572).

In [8]:
# Toxic - Britney Spears
# Yellow Submarine - The Beatles
# Stairway to Heaven - Led Zeppelin
# Wonderwall - Oasis
# It's my life - Bon Jovi
# Californication - Red Hot Chilli Peppers
# Helena - My Chemical Romance

song_name = input('Choose a song: ')
results = sp.search(q=f'track:{song_name}', limit=1)
track_id = results['tracks']['items'][0]['id']
audio_features = sp.audio_features(track_id)

df_ = pd.DataFrame(audio_features)
new_features = df_[x.columns]

scaled_x = scaler.transform(new_features)
cluster = kmeans.predict(scaled_x)

filtered_df = scaled_df[scaled_df['cluster'] == cluster[0]][x.columns]
closest, _ = pairwise_distances_argmin_min(scaled_x, filtered_df)

Choose a song: Californication - Red Hot Chilli Peppers


In [9]:
scaled_df.loc[closest[0]]['song name'], scaled_df.loc[closest[0]]['artist']

('Anyone', 'Justin Bieber')

#### Next Steps:

In [10]:
# put everything inside a function
def recommend_song():
    # get song id
    song_name = input('Choose a song: ')
    results = sp.search(q=f'track:{song_name}', limit=1)
    track_id = results['tracks']['items'][0]['id']
    # get song features with the obtained id
    audio_features = sp.audio_features(track_id)
    # create dataframe
    df_ = pd.DataFrame(audio_features)
    new_features = df_[x.columns]
    # scale features
    scaled_x = scaler.transform(new_features)
    # predict cluster
    cluster = kmeans.predict(scaled_x)
    # filter dataset to predicted cluster
    filtered_df = scaled_df[scaled_df['cluster'] == cluster[0]][x.columns]
    # get closest song from filtered dataset
    closest, _ = pairwise_distances_argmin_min(scaled_x, filtered_df)
    # return it in a readable way
    print('\n [RECOMMENDED SONG]')
    return ' - '.join([scaled_df.loc[closest]['song name'].values[0], scaled_df.loc[closest]['artist'].values[0]])

In [12]:
recommend_song()

Choose a song: Californication - Red Hot Chilli Peppers

 [RECOMMENDED SONG]


'Anyone - Justin Bieber'

![](https://c.tenor.com/GYswjo8j1QsAAAAC/dungeons-dragons-80s.gif)